In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re
import numpy as np
import pandas as pd
from tqdm import tqdm


In [ ]:
list_url = 'https://www.parliament.gov.sg/mps/list-of-current-mps'
r_list = requests.get(list_url)
doc = bs(r_list.text)

In [ ]:
listmps_raw = doc.find_all('div',{'class':'col-md-8 col-xs-12 mp-sort-name'})
mp_list = [ref.find('a')['href'] for ref in listmps_raw]

In [ ]:
def get_MPdetails(slug):
    profile={}
    BASE_URL = "https://www.parliament.gov.sg"
    r = requests.get(BASE_URL + slug)
    soup = bs(r.text, 'lxml')
    try:
        profile['name']=soup.find('h1',{'class':'indv-mp-header-mobile'}).text
        profile['designation'] = soup.find('div',{'class':'row mp-designation-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).text.split('\t')[0].replace('\r\n','')
        if profile['designation'] == 'Nominated Member of Parliament':
            profile['constituency'] = 'NA'
            profile['party'] ='NA'
        elif profile['designation'] == 'Non-Constituency Member of Parliament':
            profile['constituency'] = 'NA'
            profile['party'] = soup.find('div',{'class':'row mp-party-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).text.strip()
        else: 
            profile['constituency'] = soup.find('div', {'class':'row mp-constituency-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).get_text().split('\n')[1].strip()
            profile['party'] = soup.find('div',{'class':'row mp-party-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).text.strip()
            
    except:
        print("URL broke at {BASE_URL + uri}")
        
    return profile

    

In [ ]:
result_list = []
for slug in mp_list: 
    result_list.append(get_MPdetails(slug))
    

In [ ]:
MPs = pd.DataFrame(result_list, columns = ['name','designation','constituency','party'])

In [ ]:
MPs.to_csv('MPs.csv', index= False)